# 系统评估与优化

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableBranch, RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_community.vectorstores import Chroma

from dotenv import load_dotenv, find_dotenv

from ark_embedding import ArkEmbeddings

import os

In [12]:
_ = load_dotenv(find_dotenv('.env.local'))

# 设置大模型参数
api_key = os.getenv("DEEPSEEK_API_KEY")
api_url = os.getenv("DEEPSEEK_API_URL")
model = os.getenv("DEEPSEEK_MODEL")
# 设置向量数据库参数
embedding_api_key = os.getenv("ARK_API_KEY")
embedding_api_url = os.getenv("ARK_API_URL")
embedding_model = os.getenv("ARK_EMBEDDING_MODEL")

In [13]:
# 初始化 Embeddings
embedding = ArkEmbeddings(
    api_key=embedding_api_key,
    api_url=embedding_api_url,
    model=embedding_model,
)

# 向量数据库持久化路径
persist_directory = "../data_base/vector_db/chroma"

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
)

# 将向量数据库转换为检索器
retriever = vectordb.as_retriever()

llm = ChatOpenAI(
        openai_api_key=api_key,
        base_url=api_url,
        model_name=model,
        temperature=0.0,
    )

### 对比不同prompt的输出

In [14]:
template_v1 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(template=template_v1)

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrievel_chain = retriever | RunnableLambda(combine_docs)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | {
        "answer": QA_CHAIN_PROMPT | llm | StrOutputParser(),
        "context": lambda x: x["context"]
    }
)

# 测试效果
print("问题一：")
question = "南瓜书和西瓜书有什么关系？"
result = qa_chain.invoke(question)
print(result["answer"])

print("问题二：")
question = "应该如何使用南瓜书？"
result = qa_chain.invoke(question)
print(result["answer"])

问题一：
南瓜书是对西瓜书（《机器学习》）中公式的详细解释和推导，旨在帮助读者更好地理解西瓜书的内容。谢谢你的提问！
问题二：
可以从GitHub获取最新版PDF（地址已提供），扫描二维码加入读者交流群获取帮助。版权声明中规定了使用许可方式。谢谢你的提问！


In [15]:
template_v2 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(template=template_v2)

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrievel_chain = retriever | RunnableLambda(combine_docs)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | {
        "answer": QA_CHAIN_PROMPT | llm | StrOutputParser(),
        "context": lambda x: x["context"]
    }
)

# 测试效果
print("问题一：")
question = "南瓜书和西瓜书有什么关系？"
result = qa_chain.invoke(question)
print(result["answer"])

print("问题二：")
question = "应该如何使用南瓜书？"
result = qa_chain.invoke(question)
print(result["answer"])

问题一：
南瓜书是《机器学习公式详解》（即南瓜书）的简称，它是作为周志华教授《机器学习》（即西瓜书）的配套学习资料而编写的。南瓜书的主要目的是帮助读者更好地理解和掌握西瓜书中的数学公式和推导过程，消除阅读过程中的“数学恐惧”。具体来说：

1. **内容关联**：南瓜书对西瓜书中的公式进行了详细的解释和推导，例如在提供的上下文中提到的第12章和第13章的内容，以及第5章关于深度学习的部分。

2. **目标读者**：南瓜书的目标读者是那些已经学习过《高等数学》、《线性代数》和《概率论与数理统计》的读者，帮助他们理解西瓜书中复杂的数学内容。

3. **补充说明**：南瓜书不仅解释了西瓜书中的公式，还补充了一些背景知识和额外的说明，例如对“算法”和“模型”的进一步解释，以及对深度学习概念的详细说明。

4. **结构对应**：南瓜书的章节结构与西瓜书基本对应，例如第1章绪论、第5章深度学习等，方便读者对照学习。

总之，南瓜书是西瓜书的辅助学习资料，旨在帮助读者更轻松地掌握西瓜书中的数学内容和机器学习理论。
问题二：
根据提供的上下文，以下是关于如何使用《南瓜书》的具体信息：

### 1. **获取最新版本**
   - 最新版PDF可通过GitHub发布页获取：  
     [https://github.com/datawhalechina/pumpkin-book/releases](https://github.com/datawhalechina/pumpkin-book/releases)  
     建议定期检查更新，以获取修正或补充内容。

### 2. **版权与使用规范**
   - 本书采用**知识共享署名-非商业性使用-相同方式共享4.0国际许可协议**（CC BY-NC-SA 4.0），使用时需遵守以下条款：
     - **署名**：使用时需标明原作者及来源。
     - **非商业性**：禁止用于商业用途。
     - **相同方式共享**：若基于本书内容改编，需以相同协议发布。

### 3. **读者交流与支持**
   - 可扫描书中提供的二维码，回复关键词“南瓜书”加入读者交流群，与编委或其他读者互动，解决学习中的问题。

### 4. **内容定位**
   - 书中未明确说明具体使用场景，但根据编委团队（如Data

### 使用大模型进行评估

In [16]:
# 评估用prompt
prompt = '''
你是一个模型回答评估员。
接下来，我将给你一个问题、对应的知识片段以及模型根据知识片段对问题的回答。
请你依次评估以下维度模型回答的表现，分别给出打分：

① 知识查找正确性。评估系统给定的知识片段是否能够对问题做出回答。如果知识片段不能做出回答，打分为0；如果知识片段可以做出回答，打分为1。

② 回答一致性。评估系统的回答是否针对用户问题展开，是否有偏题、错误理解题意的情况，打分分值在0~1之间，0为完全偏题，1为完全切题。

③ 回答幻觉比例。该维度需要综合系统回答与查找到的知识片段，评估系统的回答是否出现幻觉，打分分值在0~1之间,0为全部是模型幻觉，1为没有任何幻觉。

④ 回答正确性。该维度评估系统回答是否正确，是否充分解答了用户问题，打分分值在0~1之间，0为完全不正确，1为完全正确。

⑤ 逻辑性。该维度评估系统回答是否逻辑连贯，是否出现前后冲突、逻辑混乱的情况。打分分值在0~1之间，0为逻辑完全混乱，1为完全没有逻辑问题。

⑥ 通顺性。该维度评估系统回答是否通顺、合乎语法。打分分值在0~1之间，0为语句完全不通顺，1为语句完全通顺没有任何语法问题。

⑦ 智能性。该维度评估系统回答是否拟人化、智能化，是否能充分让用户混淆人工回答与智能回答。打分分值在0~1之间，0为非常明显的模型回答，1为与人工回答高度一致。

你应该是比较严苛的评估员，很少给出满分的高评估。
用户问题：
```
{}
```
待评估的回答：
```
{}
```
给定的知识片段：
```
{}
```
你应该返回给我一个可直接解析的 Python 字典，字典的键是如上维度，值是每一个维度对应的评估打分。
不要输出任何其他内容。
'''

In [17]:
# 做一些简单的封装
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"), 
    base_url=os.getenv("DEEPSEEK_API_URL")
)

def get_gpt_messages(prompt):
    '''
    构造 GPT 模型请求参数 messages

    请求参数：
        prompt: 对应的用户提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages

def get_completion(prompt, model="deepseek-chat", temperature=0.):
    '''
    获取 GPT 模型调用结果

    请示参数：
        prompt: 对应的提示词
        model: 调用的模型，默认为 deepseek-chat, 也可以按需选择其它模型
        temperature: 模型输出的温度系数, 控制输出的随机程度, 取值范围为 [0, 2.0], 温度系数越低, 输出内容越一致。
    '''
    response = client.chat.completions.create(
        model=model,
        messages=get_gpt_messages(prompt),
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

In [19]:
# 生成回答
question = "应该如何使用南瓜书？"
result = qa_chain.invoke(question)
answer = result["answer"]
knowledge = result["context"]
print("-"*100)
print(question)
print("-"*100)
print(answer)
print("-"*100)
print(knowledge)

----------------------------------------------------------------------------------------------------
应该如何使用南瓜书？
----------------------------------------------------------------------------------------------------
根据提供的上下文，以下是关于如何使用南瓜书的详细说明：

### 1. **获取最新版本**
- 访问GitHub发布页：通过链接 [https://github.com/datawhalechina/pumpkin-book/releases](https://github.com/datawhalechina/pumpkin-book/releases) 下载最新版PDF文件。

### 2. **版权与使用规范**
- **许可协议**：南瓜书采用 **知识共享署名-非商业性使用-相同方式共享4.0国际许可协议**（CC BY-NC-SA 4.0）。  
  - 允许：自由分享、改编，但需**署名作者**，且**不得用于商业用途**。  
  - 衍生作品需以**相同协议**发布。

### 3. **加入读者社区**
- **交流群**：扫描上下文中的二维码，回复关键词“南瓜书”加入读者群，可与其他学习者讨论或获取帮助。

### 4. **致谢与贡献**
- 书中列有早期贡献者名单（如awyd234、feijuan等），若需引用或二次创作，请确保遵守协议中的署名要求。

### 注意事项
- 上下文未提及其具体内容或学习路径，建议结合原书目录或官方指南进一步规划使用方式。  
- 商业用途需额外授权，否则可能违反许可协议。  

如需更详细的使用场景说明（如教学、笔记整理等），建议参考书内前言或联系编委会（主编Sm1les等）确认。
----------------------------------------------------------------------------------------------------
←_←

←_←

5.1 综合样例

最新版PDF 获取地址：https://github.com/dat

In [18]:
# 生成评估结果
prompt = prompt.format(question, answer, knowledge)
result = get_completion(prompt)
print(result)

```python
{
    "知识查找正确性": 1,
    "回答一致性": 0.9,
    "回答幻觉比例": 0.8,
    "回答正确性": 0.9,
    "逻辑性": 1,
    "通顺性": 1,
    "智能性": 0.8
}
```


## 效果优化

In [20]:
_ = load_dotenv(find_dotenv('.env.local'))

# 设置大模型参数
api_key = os.getenv("DEEPSEEK_API_KEY")
api_url = os.getenv("DEEPSEEK_API_URL")
model = os.getenv("DEEPSEEK_MODEL")
# 设置向量数据库参数
embedding_api_key = os.getenv("ARK_API_KEY")
embedding_api_url = os.getenv("ARK_API_URL")
embedding_model = os.getenv("ARK_EMBEDDING_MODEL")

In [21]:
# 初始化 Embeddings
embedding = ArkEmbeddings(
    api_key=embedding_api_key,
    api_url=embedding_api_url,
    model=embedding_model,
)

# 向量数据库持久化路径
persist_directory = "../data_base/vector_db/chroma"

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
)

# 将向量数据库转换为检索器
retriever = vectordb.as_retriever()

llm = ChatOpenAI(
        openai_api_key=api_key,
        base_url=api_url,
        model_name=model,
        temperature=0.0,
    )

In [22]:
template_v1 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(template=template_v1)

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrievel_chain = retriever | RunnableLambda(combine_docs)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | {
        "answer": QA_CHAIN_PROMPT | llm | StrOutputParser(),
        "context": lambda x: x["context"]
    }
)

In [23]:
# 测试效果
question = "什么是南瓜书？"
result = qa_chain.invoke(question)
print(result["answer"])

南瓜书是一本由datawhalechina团队编写的开源书籍，最新版PDF可在GitHub上获取。它由多位编委共同完成，并采用知识共享许可协议发布。谢谢你的提问！


In [24]:
template_v2 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(template=template_v2)

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrievel_chain = retriever | RunnableLambda(combine_docs)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | {
        "answer": QA_CHAIN_PROMPT | llm | StrOutputParser(),
        "context": lambda x: x["context"]
    }
)

In [25]:
# 测试效果
question = "什么是南瓜书？"
result = qa_chain.invoke(question)
print(result["answer"])

根据提供的上下文，**南瓜书**是由Datawhale社区成员协作编写的开源项目（以PDF形式发布），其最新版可通过GitHub链接获取。以下是具体信息：

1. **内容与性质**  
   上下文未直接说明南瓜书的具体内容，但根据编委会结构、致谢名单及版权声明可推断，它是一本采用**知识共享协议**（CC BY-NC-SA 4.0）的技术类或学习资料，允许非商业性共享使用。

2. **制作与贡献**  
   - 由主编Sm1les等人牵头，多位编委和贡献者共同完成（如封面设计由林王茂盛创作）。  
   - 特别列出了早期贡献者名单（如awyd234、feijuan等），体现社区协作性质。

3. **获取与交流**  
   - 发布渠道：GitHub的Release页面提供PDF下载。  
   - 读者可通过扫描二维码加入交流群，关键词为“南瓜书”。

4. **版权信息**  
   采用**知识共享协议**，允许署名、非商业用途及相同方式共享。

如需更详细的内容介绍（如书籍主题、目录等），建议直接查阅提供的GitHub链接或加入读者群询问。


In [27]:
question = "使用大模型时，构造 Prompt 的原则有哪些"
result = qa_chain.invoke(question)
print(result["answer"])

根据上下文内容，构造Prompt的核心原则可归纳为以下两大点，并附具体实施技巧：

---

### **一、编写清晰具体的指令**
1. **类比外星人沟通原则**  
   假设模型对背景知识一无所知，需提供：
   - 明确的意图表达（避免歧义）
   - 丰富的上下文细节（如任务目标、格式要求、示例等）
   - 长而结构化的Prompt往往优于简短模糊的指令

2. **关键技巧**  
   - **使用分隔符**：通过```、"""、XML标签等清晰划分输入的不同部分（如指令与数据）。  
   - **结构化输出**：指定响应格式（如JSON、列表、分步骤回答）。  
   - **示例引导**：提供输入-输出样例，帮助模型理解任务模式。

---

### **二、给予模型充足思考时间**
1. **分步推理要求**  
   - 明确要求模型“逐步分析”或“分步骤解答”，避免直接输出结论。  
   - 例如：“请先解释问题背景，再列出解决步骤，最后给出最终答案。”

2. **延迟生成结论**  
   - 在复杂任务中，可设计多轮交互Prompt，逐步引导模型深入推理。

---

### **补充说明**
- **迭代优化**：Prompt设计需通过多次调整测试（如Jupyter Notebook中修改观察输出），逐步逼近最优解。  
- **幻觉问题**：需意识到模型可能生成虚假内容（截至2023年7月），可通过Prompt约束和后期验证缓解。  
- **换行符处理**：反斜杠`\`仅用于文本排版，实际调用时需注意不同模型对换行符`\n`的敏感性差异。

---

**总结**：成功的Prompt设计需平衡“清晰度”与“思考空间”，并通过实践迭代优化。开发者应像指导新手一样提供详尽引导，同时避免过度限制模型的推理过程。


In [28]:
template_v3 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(template=template_v3)

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrievel_chain = retriever | RunnableLambda(combine_docs)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | {
        "answer": QA_CHAIN_PROMPT | llm | StrOutputParser(),
        "context": lambda x: x["context"]
    }
)

In [29]:
question = "使用大模型时，构造 Prompt 的原则有哪些"
result = qa_chain.invoke(question)
print(result["answer"])

使用大模型时，构造 Prompt 的核心原则和技巧可总结如下，分为两大核心原则及具体实践方法：

---

### **一、两大核心原则**
1. **编写清晰、具体的指令**  
   - **类比**：像向一个毫无人类常识的外星人解释任务，需避免歧义，提供充分上下文。  
   - **关键点**：  
     - 更长的 Prompt 可能更有效，因细节和上下文能帮助模型更准确理解意图。  
     - 避免过于简略的指令，否则模型可能无法捕捉任务细节。  

2. **给予模型充足思考时间**  
   - **类比**：类似人类解题，仓促回答易出错，需逐步推理。  
   - **关键点**：  
     - 在 Prompt 中要求模型分步骤思考（如“请逐步分析”）。  
     - 避免直接要求即时输出复杂答案，允许模型“消化”问题。

---

### **二、具体实践技巧**
1. **使用分隔符区分不同部分**  
   - 例如用 ```、`"`、`---` 等符号分隔指令、输入数据和上下文，避免混淆。  
   - *示例*：  
     ```
     请翻译以下文本（用引号标注的部分）：
     "Hello, world!"
     ```

2. **提供结构化示例**  
   - 对于复杂任务，给出输入-输出的示范样本，明确格式要求。  
   - *示例*：  
     ```
     任务：将句子分类为积极/消极。  
     示例输入："这个产品很好用。" → 输出：积极  
     ```

3. **逐步引导推理**  
   - 对需要逻辑的任务，明确要求分步骤解答。  
   - *示例*：  
     ```
     问题：计算圆的面积，半径为5。  
     步骤1：回忆公式（面积=πr²）。  
     步骤2：代入数值计算。
     ```

4. **多轮迭代优化 Prompt**  
   - 通过测试不同版本 Prompt 的效果，逐步调整语言、细节或结构。  
   - *流程*：  
     - 初版 Prompt → 观察输出 → 修正模糊点 → 重复直至满意。  
     - 复杂场景需在多样本上评估平均表现。

5. **注意技术细节**  
   - 换

#### 标明知识来源

In [30]:
template_v4 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体。
请你附上回答的来源原文，以保证回答的正确性。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(template=template_v4)

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrievel_chain = retriever | RunnableLambda(combine_docs)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | {
        "answer": QA_CHAIN_PROMPT | llm | StrOutputParser(),
        "context": lambda x: x["context"]
    }
)

In [32]:
question = "强化学习的定义是什么"
result = qa_chain.invoke(question)
print(result["answer"])

根据提供的上下文，书中第16章标题为“强化学习”，但具体内容仅显示节标题“16.1 任务与奖赏”（页码180），未包含强化学习的明确定义。因此，基于当前提供的上下文无法回答该问题。

来源原文引用：  
```
第16 章强化学习  
180  
16.1 任务与奖赏. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  
180  
```

建议：若需准确定义，请提供更多上下文（如16.1节的具体内容）或参考其他权威资料。


#### 构造思维链

In [33]:
question = "我们应该如何去构造一个LLM项目"
result = qa_chain.invoke(question)
print(result["answer"])

根据提供的上下文，构造一个LLM项目的核心方法可总结为以下步骤，并附具体依据：

1. **迭代式Prompt开发**  
   - 先编写初版Prompt，通过多轮调整逐步优化，而非追求初始完美。  
   - 复杂项目需在多个样本上迭代训练，评估平均表现（来源："具体来说，首先编写初版 Prompt...平均表现"）。  

2. **分阶段优化**  
   - 初期：快速验证Prompt基础效果。  
   - 成熟期：在多样本集上细致评估性能（需较高计算资源）（来源："在应用较为成熟后...细致优化"）。  

3. **实践验证工具**  
   - 使用Jupyter Notebook实时修改Prompt并观察输出差异，理解优化过程（来源："读者可以在Jupyter Notebook上...实践准备"）。  

4. **API集成开发**  
   - 通过API调用LLM（如GPT-3）构建应用，而非仅限Web界面的一次性任务（来源："对于开发人员...构建软件应用程序"）。  

5. **缓解模型缺陷**  
   - 针对幻觉问题，需结合Prompt优化等措施提升可靠性（来源："语言模型的幻觉问题...予以缓解"）。  

**关键原则**：  
- 优先快速迭代而非完美设计，通过实验调整找到可靠方案（来源："Prompt 工程师的核心...正确方法"）。  
- 开发中需注意技术细节（如反斜杠/换行符对模型的影响）（来源："关于反斜杠使用的说明"）。  

完整流程示例：  
1. 设计初版Prompt → 2. 单样本测试 → 3. 多轮调整 → 4. 多样本评估 → 5. API集成 → 6. 部署后持续优化。


In [34]:
template_v5 = """
请你依次执行以下步骤：
① 使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。
你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体。
上下文：
{context}
问题: 
{question}
有用的回答:
② 基于提供的上下文，反思回答中有没有不正确或不是基于上下文得到的内容，如果有，回答你不知道
确保你执行了每一个步骤，不要跳过任意一个步骤。
"""

QA_CHAIN_PROMPT = PromptTemplate(template=template_v5)

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrievel_chain = retriever | RunnableLambda(combine_docs)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | {
        "answer": QA_CHAIN_PROMPT | llm | StrOutputParser(),
        "context": lambda x: x["context"]
    }
)

In [35]:
question = "我们应该如何去构造一个LLM项目"
result = qa_chain.invoke(question)
print(result["answer"])

### 如何构造一个LLM项目（基于上下文的分析）  

#### 1. **迭代式Prompt开发**  
   - **初版Prompt设计**：首先编写基础Prompt，明确任务目标（如文本总结、推理等）。  
   - **多轮调整**：通过反复修改Prompt并观察输出，逐步优化效果。例如，在Jupyter Notebook中实践示例，对比不同Prompt的差异。  
   - **复杂场景优化**：对于成熟应用，需在多样本集上评估Prompt性能，但需注意计算资源消耗（上下文5.1部分）。  

#### 2. **核心开发原则**  
   - **接受不完美**：Prompt工程师的目标是通过试错找到可靠方案，而非追求初始完美（上下文“总之”段落）。  
   - **实践导向**：建议动手修改Prompt并观察输出，以深入理解优化过程（上下文实践建议部分）。  

#### 3. **应对模型缺陷**  
   - **幻觉问题**：需通过Prompt优化等措施缓解语言模型的不可靠性，确保应用安全性（上下文“产品说明书”部分）。  

#### 4. **技术细节注意**  
   - **分隔符使用**：清晰划分输入部分（如指令、数据），避免混淆（上下文1.1部分）。  
   - **换行符处理**：反斜杠`\`仅用于文本格式化，实际调用API时需测试换行符`\n`对模型的影响（上下文“注意”部分）。  

#### 5. **API与开发整合**  
   - **优先API调用**：LLM的强大功能在于通过API快速构建应用（如聊天机器人、文本转换等），而非仅限Web界面的一次性任务（上下文英文原版Prompt部分）。  

#### 未涵盖的上下文外内容  
   - 团队协作、数据预处理、模型微调等步骤未提及，因上下文未提供相关信息。  

**反思**：以上回答均基于上下文，无编造内容。若需更全面的项目构造方法（如数据收集、部署等），需额外信息。


#### 增加一个指令解析

In [36]:
question = "LLM的分类是什么？给我返回一个 Python List"
result = qa_chain.invoke(question)
print(result["answer"])

根据提供的上下文，我无法找到关于“LLM的分类”的相关信息。上下文中主要涉及的是综合样例、图解释、半监督聚类、概率图模型、产品运输信息、用户评价处理以及一些数学公式的解释等内容，但没有提及LLM（Large Language Models）或其分类的任何信息。

因此，我的回答是：我不知道。


In [37]:
prompt_input = '''
请判断以下问题中是否包含对输出的格式要求，并按以下要求输出：
请返回给我一个可解析的Python列表，列表第一个元素是对输出的格式要求，应该是一个指令；第二个元素是去掉格式要求的问题原文
如果没有格式要求，请将第一个元素置为空
需要判断的问题：
```
{}
```
不要输出任何其他内容或格式，确保返回结果可解析。
'''

response = get_completion(prompt_input.format(question))
response

'["给我返回一个 Python List", "LLM的分类是什么？"]'

In [39]:
prompt_output = '''
请根据回答文本和输出格式要求，按照给定的格式要求对问题做出回答
需要回答的问题：
```
{}
```
回答文本：
```
{}
```
输出格式要求：
```
{}
```
'''

In [40]:
question = 'LLM的分类是什么？给我返回一个 Python List'
# 首先将格式要求与问题拆分
input_lst_s = get_completion(prompt_input.format(question))
# 找到拆分之后列表的起始和结束字符
start_loc = input_lst_s.find('[')
end_loc = input_lst_s.find(']')
rule, new_question = eval(input_lst_s[start_loc:end_loc+1])
# 接着使用拆分后的问题调用检索链
result = qa_chain.invoke(new_question)
result_context = result
# 接着调用输出格式解析
response = get_completion(prompt_output.format(new_question, result_context, rule))
response

"['我不知道']"

### 评估并优化检索部分